###### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [18]:
movieID = pd.read_csv('ratings.csv')

In [19]:
movieID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [35]:
def rating_movie(row):
    if row.rating <= 2:
        return('низкий рейтинг')
    elif row.rating > 2 and row.rating< 4.5:
        return('средний рейтинг')
    else:
        return('высокий рейтинг')

In [36]:
movieID['class'] = movieID.apply(rating_movie, axis=1)

In [41]:
movieID.head(5)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


###### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [268]:
region = pd.read_csv('keywords.csv')

In [269]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
keyword    100000 non-null object
shows      100000 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [274]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск', 'одноклассники']
}

In [275]:
def geo_region(geo):
    for key, values in geo_data.items():
        if geo.keyword in values:
            return key
        else:
            return ('undefined')

In [276]:
region['region_geo'] = region.apply(geo_region, axis=1)

In [277]:
region.groupby('region_geo').count()

,keyword,shows
region_geo,,
undefined,99997,99997
Центр,3,3


###### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
        для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [3]:
data_ratings = pd.read_csv('ratings.csv', 'r', delimiter=',')
data_movies = pd.read_csv('movies.csv', 'r', delimiter=',')

In [4]:
def production_year(film):
    years = range(1950,2010)
    val = [yr for yr in years if str(yr) in film['title']]
    if val:
        return val[0]
    else:
        return 1900

In [5]:
data_movies['year']=data_movies.apply(production_year, axis=1)

In [6]:
#объединим две датафрейма, чтобы посчитать средний рейтинг по группировке года
joined = data_movies.merge(data_ratings, on='movieId', how='left')
jd = joined.groupby('year').rating.mean().reset_index()
jd.sort_values('rating', ascending=False)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
56,2005,3.448434
54,2003,3.444777
47,1996,3.426600
48,1997,3.415764


In [7]:
jd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
year      61 non-null int64
rating    61 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.1 KB


In [8]:
jd.head(10)

,year,rating
0,1900,3.592981
1,1950,3.915254
2,1951,3.983539
3,1952,4.000000
4,1953,3.754587
5,1954,3.994220
6,1955,3.751724
7,1956,3.617647
8,1957,4.014241
9,1958,3.823171
